# Robot Motion Planning

## Function Specifications

Goal is to find a shortest path through a maze/grid in which there are hurdles as well. Grid/Maze format:
- 0 = Navigable space
- 1 = Occupied space

Define a function, search() that returns a list in the form of [optimal path length, row, col]. For the grid shown below, function should output [11, 4, 5].

If there is no valid path from the start point to the goal, function should return the string 'fail'.

In [1]:
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, function should return the string
# 'fail'
# ----------

# Grid/Maze format:
#   0 = Navigable space
#   1 = Occupied space

grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 0]]

init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

def search(grid,init,goal,cost):
    # ----------------------------------------
    # insert code here
    # ----------------------------------------

    return path

In [35]:
def get_next_cell(current_cell, move):
    next_cell = [current_cell[0] + move[0], current_cell[1] + move[1]]
    return next_cell

In [36]:
#Function to check if making the passed move is possible and valid
def is_move_valid(grid, current_cell, move):
    #calculate the next cell
    next_cell = get_next_cell(current_cell, move)
    
    #make sure we are not overshooting the maze/grid boundary
    if next_cell[0] >= len(grid[0]) or next_cell[1] >= len(grid[1]):
        return False
    
    #make sure we are not undershooting the maze/grid boundary
    if next_cell[0] < 0 or next_cell[1] < 0:
        return False
    
    #make sure the cell is not already occupied, either because 
    #it is an obstacle or we traveresed it before
    if grid[next_cell[0]][next_cell[1]] == 1:
        return False
    
    return True

In [37]:
#function to make a move, it assumes that the move
#passed is a valid move to make
def make_move(grid, current_cell, move):
    #calculate the next cell
    #calculate the next cell
    next_cell = get_next_cell(current_cell, move)
    
    #update the grid cell as marked
    grid[next_cell[0]][next_cell[1]] = 1

In [38]:
#function to find possible adjacent cells which are not occupied
def find_available_cells(grid, current_cell, moves):
    available_cells = []
    
    for move in moves:
        if (is_move_valid(grid, current_cell, move)):
            available_cells.append(get_next_cell(current_cell, move))
            
    return available_cells

In [41]:
# is_move_valid(grid, init, delta[3])
# make_move(grid, init, delta[3])
find_available_cells(grid, init, delta)
# print(grid)
# print(delta[0])
# print(grid[0][1])


[[1, 0]]

In [23]:
def test(c):
    c = c + 10

v = 10
test(v)
print(v)

10


In [31]:
a = []
a.append([1,1])
a.append(1)
a.remove(1)
print(a)

[[1, 1]]
